In [12]:
import pandas as pd
import numpy as np

def table_tonav(df):
    """Превращает таблицу, где первый столбец маги, а остальные нормы по тпг в таблицу для загрузки этих норм в нав. 
    По умолчанию максимальное значение 1, коэф отставания 0 индекс = номер магазина"""    
    
    norms_list = df.T.to_numpy().tolist()
    
    tpg_list = df.columns.tolist()
    
#добавим бк/иф к названиям тпг   
    new_tpg_list = ['БК_' + str(tpg) if int(str(tpg)[:4]) >= 1500 else 'ИФ_' + str(tpg) for tpg in tpg_list]
          
    tpg = [x for x in new_tpg_list for i in range(len(df.index.tolist()))]
    shop = df.index.tolist()*len(df.columns.tolist())
    norms = [round(item) for sublist in norms_list for item in sublist]#Все нормы округляются
    data = {'Товарная Группа':tpg,'Норма':norms,'Макс. допустимое кол-во на точке':[1]*len(norms),'коэф. Корр отставания проведения продаж':[0]*len(norms)}
    new_df = pd.DataFrame(index=shop, data=data)
    new_df.index.name = 'Код'                    
    return new_df

def priority_tonav(df):
    
#индекс = номер магазина    
    norms_list = df.T.to_numpy().tolist()
    tpg_list = df.columns.tolist()
    tpg = [x for x in tpg_list for i in range(len(df.index.tolist()))]
    shop = df.index.tolist()*len(df.columns.tolist())
    norms = [item for sublist in norms_list for item in sublist]#Все нормы округляются
    data = {'Товарное направление':tpg,'Приоритет':[100]*len(norms),'Повышающий коэффициент':[1]*len(norms),'Исключить из распределения':norms}
    new_df = pd.DataFrame(index=shop, data=data)
    new_df.index.name = 'Код'                    
    return new_df

In [22]:
# любая табла с нормами по тпг
df = pd.read_excel(r'C:\Users\Dotsenko.Semen\Documents\нормы ифбк кидс 12.02.20',index_col=0)
table_tonav(df).to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\в_нав.xlsx')

In [19]:
df = pd.read_excel(r'C:\Users\Dotsenko.Semen\Documents\нормы ифбк кидс 12.02.20.xlsx',sheet_name='Стало', skiprows=1,index_col=0)


In [20]:
del df['Unnamed: 1']

In [21]:
df

,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,...,1584,1585,1586,1587,1589,1590,1591,1592,1593,1594
Товарная подгруппа,,,,,,,,,,,,,,,,,,,,,
3839,0,0,0,0,0,0,0,0,0,0,...,0,0,5.977246,0,23.908983,32.874851,11.954491,6.973453,0.000000,0.000000
4667,0,0,0,0,0,0,0,0,0,0,...,0,0,12.950699,0,23.908983,47.817966,11.954491,11.954491,9.962076,4.981038
4743,0,0,0,0,0,0,0,0,0,0,...,0,0,5.965697,0,23.862787,32.811332,11.931393,6.959979,0.000000,0.000000
4764,0,0,0,0,0,0,0,0,0,0,...,0,0,5.965697,0,23.862787,32.811332,11.931393,6.959979,0.000000,0.000000
4765,0,0,0,0,0,0,0,0,0,0,...,0,0,5.965697,0,23.862787,32.811332,11.931393,6.959979,0.000000,0.000000
4782,0,0,0,0,0,0,0,0,0,0,...,0,0,5.965697,0,23.862787,32.811332,11.931393,6.959979,0.000000,0.000000


In [22]:
table_tonav(df).to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\в_нав.xlsx')

In [6]:
# ОБРУЧИ

#загружаем нормы по размерам и по тпг
wed_rings1 = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\матрица обручей.xlsx',
    sheet_name='гладкая_плоская размеры_тпг',index_col=0,skiprows=3)
del wed_rings1['адрес']
del wed_rings1['норма']

ring_cols = [0, 3, 8] + list(range(12,25))
wed_rings2 = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\матрица обручей.xlsx',
    sheet_name='Матрица по тпг',index_col=0,skiprows=3, usecols=ring_cols)

#применяем функцию исправляем максимумы соединяем в один датафрейм и сохраняем
wed_rings_tonav1 = table_tonav(wed_rings1)
wed_rings_tonav1['Макс. допустимое кол-во на точке'] = wed_rings_tonav1['Норма']
wed_rings_tonav2 = table_tonav(wed_rings2)

wed_rings_tonav = pd.concat([wed_rings_tonav1, wed_rings_tonav2 ])

wed_rings_tonav.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\в_нав.xlsx')

In [11]:
#МУСУЛЬМАНКА
from_26 = [0] + list(range(26,49))
mus_df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\мусульманкэ.xlsx',
    sheet_name='нормы культ',index_col=0,skiprows=2,usecols=from_26)

mus_df.columns = [x[:4] for x in mus_df.columns.tolist()]

mus_df = table_tonav(mus_df)

mus_df.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\мус_внав.xlsx',
sheet_name='Лист1')

In [7]:
# ПО ТГ

#рассчитываем номера столбцов для каждого ТН

if_string_lenth = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='ИФ',nrows=0,skiprows=2)

lst = ['ИФ СЕРЬГИ' if 'ИФ СЕРЬГИ' in str(x) else x for x in if_string_lenth.columns.tolist()]

if_end = len(lst)-lst[::-1].index('ИФ СЕРЬГИ')
if_start = if_end-6
if_columnrange = [1,2] + list(range(if_start,if_end))

bk_string_lenth = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='БК',nrows=0,skiprows=2)

lst = ['БК СЕРЬГИ' if 'БК СЕРЬГИ' in str(x) else x for x in bk_string_lenth.columns.tolist()]

bk_end = len(lst)-lst[::-1].index('БК СЕРЬГИ')
bk_start = bk_end-6
bk_columnrange = [1,2] + list(range(bk_start,bk_end))


# загружаем датафреймы
begin = [1,2]
if_df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='ИФ',skiprows=3,usecols=if_columnrange,index_col=1, names=['блок','СКЛАД',
        'ИФ КОЛЬЦА','ИФ КОЛЬЦА ОБРУЧ','ИФ ПЕЧАТКИ','ИФ ПОДВЕС ДЕКОР','ИФ ПОДВЕС КУЛЬТ','ИФ СЕРЬГИ'])

bk_df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='БК',skiprows=3,usecols=bk_columnrange,index_col=1, names=['блок','СКЛАД',
        'БК КОЛЬЦА','БК КОЛЬЦА ОБРУЧ','БК ПЕЧАТКИ','БК ПОДВЕС ДЕКОР','БК ПОДВЕС КУЛЬТ','БК СЕРЬГИ'])

# убираем закрытые маги
if_df = if_df[(if_df.блок != 'з')]
del if_df['блок']

bk_df = bk_df[(bk_df.блок != 'з')]
del bk_df['блок']

#переворачиваем в нужный формат, соединяем и сохраняем
if_tonav = priority_tonav(if_df)
if_tonav.index.name = 'Код'

bk_tonav = priority_tonav(bk_df)
bk_tonav.index.name = 'Код'

total = if_tonav.append(bk_tonav)

total.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\приоритеты_внав.xlsx',
sheet_name='Лист1')


In [11]:
import pandas as pd
#пРИОРИТЕТЫ ОБРУЧИ В НАВ
path = 'C:/Остатки/болванка два.xlsx'
df = pd.read_excel(path, sheet_name='наполняха', skiprows=3,usecols=[1,2],names=['Код','Исключить из распределения'])
df['Товарное направление'], df['Приоритет'], df['Повышающий коэффициент'] = ['БК КОЛЬЦА ОБРУЧ', 100, 1]
df.set_index('Код', inplace=True)
order = ['Товарное направление', 'Приоритет', 'Повышающий коэффициент', 'Исключить из распределения']
df[order].to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\приоритеты_внав.xlsx',
sheet_name='Лист1')